In [1]:
#  learning_rate, n_estimator 영향 제일 많이 끼치는 하이퍼파라미터

In [2]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score

In [3]:
from datetime import datetime
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier # decision tree 앙상블 모델, 부스팅

from sklearn.model_selection import GridSearchCV

from sklearn.dummy import DummyClassifier

# 모델 평가 지표 scoring metrics
from sklearn.model_selection import cross_val_score # model 검증

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import jaccard_score

import ast # convert string to function
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import numpy as np
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
import random

2022-06-25 17:20:42.056648: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [4]:
## 함수
def mk_cd(x):
    if type(x) == str:
        if x.isalpha():
            return x
        else:
            if len(x) == 6:
                return x
            else:
                return "0"*(6-len(x))+x
    elif type(x) == int:
        x = str(x)
        return "0"*(6-len(x))+x

In [5]:
len_a2012 = 107935
len_a2013 = 93840
len_a2014 = 95437
len_a2015 = 158287
len_a2016 = 134332
len_a2017 = 110782
len_a2018 = 159382
len_a2019 = 133987
len_a2020 = 222267
len_a2021 = 223622

In [6]:
len1215 = len_a2012+len_a2013+len_a2014+len_a2015 # y:2015
len1316 = len1215+len_a2016 # y:2016
len1417 = len1316+len_a2017 # y:2017
len1518 = len1417+len_a2018 # y:2018
len1619 = len1518+len_a2019 # y:2019
len1720 = len1619+len_a2020 # y:2020
len1821 = len1720+len_a2021 # y:2021

In [7]:
slicing_dic = {'1215':[0, len1215, len_a2015],
               '1316':[len_a2012, len1316, len_a2016],
               '1417':[len_a2012+len_a2013, len1417, len_a2017],
               '1518':[len_a2012+len_a2013+len_a2014, len1518, len_a2018],
               '1619':[len_a2012+len_a2013+len_a2014+len_a2015, len1619, len_a2019],
               '1720':[len_a2012+len_a2013+len_a2014+len_a2015+len_a2016,len1720, len_a2020],
               '1821':[len_a2012+len_a2013+len_a2014+len_a2015+len_a2016+len_a2017,len1821, len_a2021]}

In [8]:
d1221 = pd.read_feather('../kelly/d1221_10thr.ftr', columns = None, use_threads = True)
d1221['code'] = list(map(mk_cd, d1221['code'])) # mk_cd함수 적용시켜야 ftr만들 때 에러 안남.
# NextChange
d1221['NextChange'] = np.where(d1221['NextChange']>0.05, 1, 0)

In [9]:
DecisionTree = DecisionTreeClassifier()
DecisionTree.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

In [10]:
## RandomizedSearch의 각 하이퍼파라미터의 설명

clf = RandomizedSearchCV(
    pipe,                       #파이프라인으로 학습된 모델 
    param_distributions=dists,  #하이퍼파라미터 튜닝 값으로 진행한다
    n_iter=50,                  # 반복 횟수
    cv=3,                       # 교차검증 횟수 (==> n_iter * cv 의 숫자만큼 진행됨)
    scoring='neg_mean_absolute_error', # MAE
    verbose=1,                  # 훈련 중지여부를 화면에 출력 (1= progress bar/2= one line per epoch)
    n_jobs=-1)                  # -1: 모든 CPU 사용


NameError: name 'pipe' is not defined

In [10]:
X = d1221.iloc[:,2:-1] # 1221 데이터
y = d1221.iloc[:,-1]
y.value_counts()
785633/(785633+785633)

0    785677
1    654194
Name: NextChange, dtype: int64

0.5

# GaussianNB 하이퍼 파라미터 찾기. (이거 참고해)
var smoothing

In [11]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import time
start = time.time()
# X, y
X = d1221.iloc[:,2:-1] # 1221 데이터
y = d1221.iloc[:,-1]

# seed 고정
user_seed = 0
random.seed(user_seed) # seed 고정

clf = GaussianNB() ########여기에 모델이름!!!!함수로 적을것###########
random_search = {'var_smoothing': np.linspace(0.1, 1, 10)} ###### 여기에 하이퍼 파라미터랑 넣을 값 적으면 됨!!!#######


model = RandomizedSearchCV(estimator = clf, 
                           param_distributions = random_search,
                           cv =3,
                           verbose = 1,
                           random_state = 100)

for slicer in slicing_dic:
    i=0
    print(f'<<<---{slicer} start--->>>')
    print(datetime.now().strftime('%H:%M:%S'))
    tmpX = X[slicing_dic[slicer][0]:slicing_dic[slicer][1]].reset_index(drop = True) # X를 연속 4년치 꺼내서 reset_index(drop = True)
    tmpy = y[slicing_dic[slicer][0]:slicing_dic[slicer][1]].reset_index(drop = True) # y를 연속 4년치 꺼내서 reset_index(drop = True)
    tmp_all = d1221[slicing_dic[slicer][0]:slicing_dic[slicer][1]].reset_index(drop = True) # 전체데이터

    X_train = tmpX[:-slicing_dic[slicer][2]] # 기준년도 앞
    X_test = tmpX[-slicing_dic[slicer][2]:] # 기준년도 이하
    y_train = tmpy[:-slicing_dic[slicer][2]] # 기준년도 앞
    y_test = tmpy[-slicing_dic[slicer][2]:] # 기준년도 이하
    
    # train끼리 idx 같아야 하므로
    len_train = slicing_dic[slicer][1]-(slicing_dic[slicer][2]+slicing_dic[slicer][0]) # 혹은 tmp_train.shape[0]
    len(list(range(len_train)))
    train_idx = list(range(len_train))
    random.shuffle(train_idx) # 자동으로 덮어쓰기
    
    # test끼리 idx 같아야 하므로
    len_train_to_test = slicing_dic[slicer][1]-slicing_dic[slicer][0]
    test_idx = list(range(len_train,len_train_to_test)) # +1 안해줘도 되는 거 위 cell에서 확인
    len(test_idx)
    random.shuffle(test_idx) # 자동으로 덮어쓰기
    
    X_train = X_train.loc[train_idx]
    y_train = y_train.loc[train_idx]
    X_test = X_test.loc[test_idx]
    y_test = y_test.loc[test_idx]
    ###########################################
    y_train = np.array(y_train).reshape(-1,1) # 1열짜리로 만드는 것
    y_test = np.array(y_test).reshape(-1,1)
    model.fit(X_train,y_train)
    
    predictiontree = model.best_estimator_.predict(X_test)
#     print(confusion_matrix(y_test,predictiontree))
    print(classification_report(y_test,predictiontree))
    print('best :', model.best_estimator_)
    acc3=accuracy_score(y_test,predictiontree)
    print(f'---------{i}번째 search---------: ',acc3)
    i+=1
time.time()-start

<<<---1215 start--->>>
17:21:12


297212

158287

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3, estimator=GaussianNB(),
                   param_distributions={'var_smoothing': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])},
                   random_state=100, verbose=1)

...

223622

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3, estimator=GaussianNB(),
                   param_distributions={'var_smoothing': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])},
                   random_state=100, verbose=1)

              precision    recall  f1-score   support

           0       0.62      0.00      0.00    123556
           1       0.45      1.00      0.62    100066

    accuracy                           0.45    223622
   macro avg       0.53      0.50      0.31    223622
weighted avg       0.54      0.45      0.28    223622

best : GaussianNB(var_smoothing=0.30000000000000004)
---------0번째 search---------:  0.44778689037751207


670.5186607837677

In [ ]:
gau_list = [[' GaussianNB(var_smoothing=0.7000000000000001)',0.45821829967085104],
           ['GaussianNB(var_smoothing=0.2)',0.45513354971265224],
           ['GaussianNB(var_smoothing=0.2)', 0.4465707425394017],
           ['GaussianNB(var_smoothing=0.9)',0.4512115546297574],
           ['GaussianNB(var_smoothing=0.1)', 0.4478941986909178],
           ['GaussianNB(var_smoothing=0.4)',0.46749180040221894],
           ['GaussianNB(var_smoothing=0.30000000000000004)',0.44778689037751207]]

# logistic regression 파라미터
max iter, random state

In [12]:
l2=[]
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import time
start = time.time()
# X, y
X = d1221.iloc[:,2:-1] # 1221 데이터
y = d1221.iloc[:,-1]

# seed 고정
user_seed = 0
random.seed(user_seed) # seed 고정

clf = LogisticRegression()
random_search = {'max_iter': np.linspace(50, 500, 50), 
                 'random_state': [0, 100]}


model = RandomizedSearchCV(estimator = clf, 
                           param_distributions = random_search,
                           cv =3,
                           verbose = 1,
                           random_state = 100)

for slicer in slicing_dic:
    i=0
    print(f'<<<---{slicer} start--->>>')
    print(datetime.now().strftime('%H:%M:%S'))
    tmpX = X[slicing_dic[slicer][0]:slicing_dic[slicer][1]].reset_index(drop = True) # X를 연속 4년치 꺼내서 reset_index(drop = True)
    tmpy = y[slicing_dic[slicer][0]:slicing_dic[slicer][1]].reset_index(drop = True) # y를 연속 4년치 꺼내서 reset_index(drop = True)
    tmp_all = d1221[slicing_dic[slicer][0]:slicing_dic[slicer][1]].reset_index(drop = True) # 전체데이터

    X_train = tmpX[:-slicing_dic[slicer][2]] # 기준년도 앞
    X_test = tmpX[-slicing_dic[slicer][2]:] # 기준년도 이하
    y_train = tmpy[:-slicing_dic[slicer][2]] # 기준년도 앞
    y_test = tmpy[-slicing_dic[slicer][2]:] # 기준년도 이하
    
    # train끼리 idx 같아야 하므로
    len_train = slicing_dic[slicer][1]-(slicing_dic[slicer][2]+slicing_dic[slicer][0]) # 혹은 tmp_train.shape[0]
    len(list(range(len_train)))
    train_idx = list(range(len_train))
    random.shuffle(train_idx) # 자동으로 덮어쓰기
    
    # test끼리 idx 같아야 하므로
    len_train_to_test = slicing_dic[slicer][1]-slicing_dic[slicer][0]
    test_idx = list(range(len_train,len_train_to_test)) # +1 안해줘도 되는 거 위 cell에서 확인
    len(test_idx)
    random.shuffle(test_idx) # 자동으로 덮어쓰기
    
    X_train = X_train.loc[train_idx]
    y_train = y_train.loc[train_idx]
    X_test = X_test.loc[test_idx]
    y_test = y_test.loc[test_idx]
    ###########################################
    y_train = np.array(y_train).reshape(-1,1) # 1열짜리로 만드는 것
    y_test = np.array(y_test).reshape(-1,1)
    model.fit(X_train,y_train)
    
    predictiontree = model.best_estimator_.predict(X_test)
#     print(confusion_matrix(y_test,predictiontree))
#     print(classification_report(y_test,predictiontree))
    print('best :', model.best_estimator_)
    acc3=accuracy_score(y_test,predictiontree)
    print(' acc:  ',acc3)
    l2.append([i,model.best_estimator_, acc3])
    i+=1
time.time()-start

<<<---1215 start--->>>
17:33:49


297212

158287

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3, estimator=LogisticRegression(),
                   param_distributions={'max_iter': array([ 50.        ,  59.18367347,  68.36734694,  77.55102041,
        86.73469388,  95.91836735, 105.10204082, 114.28571429,
       123.46938776, 132.65306122, 141.83673469, 151.02040816,
       160.20408163, 169.3877551 , 178.57142857, 187.75510204,
       196.93877551, 206.12244898, 215.30612245, 224.48979592,
       233.6734693...
       270.40816327, 279.59183673, 288.7755102 , 297.95918367,
       307.14285714, 316.32653061, 325.51020408, 334.69387755,
       343.87755102, 353.06122449, 362.24489796, 371.42857143,
       380.6122449 , 389.79591837, 398.97959184, 408.16326531,
       417.34693878, 426.53061224, 435.71428571, 444.89795918,
       454.08163265, 463.26530612, 472.44897959, 481.63265306,
       490.81632653, 500.        ]),
                                        'random_state': [0, 100]},
                   random_state=100, verbose=1)

...

223622

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3, estimator=LogisticRegression(),
                   param_distributions={'max_iter': array([ 50.        ,  59.18367347,  68.36734694,  77.55102041,
        86.73469388,  95.91836735, 105.10204082, 114.28571429,
       123.46938776, 132.65306122, 141.83673469, 151.02040816,
       160.20408163, 169.3877551 , 178.57142857, 187.75510204,
       196.93877551, 206.12244898, 215.30612245, 224.48979592,
       233.6734693...
       270.40816327, 279.59183673, 288.7755102 , 297.95918367,
       307.14285714, 316.32653061, 325.51020408, 334.69387755,
       343.87755102, 353.06122449, 362.24489796, 371.42857143,
       380.6122449 , 389.79591837, 398.97959184, 408.16326531,
       417.34693878, 426.53061224, 435.71428571, 444.89795918,
       454.08163265, 463.26530612, 472.44897959, 481.63265306,
       490.81632653, 500.        ]),
                                        'random_state': [0, 100]},
                   random_state=100, verbose=1)

best : LogisticRegression(max_iter=141.83673469387756, random_state=100)
 acc:   0.5518061729167971


5781.384303569794

In [13]:
# logistic_best
log_list=[['LogisticRegression(max_iter=398.9795918367347, random_state=100)',0.5416932533941511],
          ['LogisticRegression(max_iter=196.9387755102041, random_state=100)',0.5448664502873478],
          ['LogisticRegression(max_iter=141.83673469387756, random_state=100)',0.5534021772490115],
          ['LogisticRegression(max_iter=141.83673469387756, random_state=100)', 0.5483492489741627],
          ['LogisticRegression(max_iter=141.83673469387756, random_state=100)',0.5505981923619455],
          ['LogisticRegression(max_iter=141.83673469387756, random_state=100)',0.5323057403933108],
          ['LogisticRegression(max_iter=141.83673469387756, random_state=100)',0.5518061729167971]]

# Random Forest 하이퍼파라미터 찾기
max leaf nodes
- decision Tree, Gradient Boosting과 공유할 예정

In [14]:
ran_list=[]
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import time
start = time.time()
# X, y
X = d1221.iloc[:,2:-1] # 1221 데이터
y = d1221.iloc[:,-1]

# seed 고정
# user_seed = 0
# random.seed(user_seed) # seed 고정

clf = RandomForestClassifier()
random_search = {'max_leaf_nodes': [4,5,6,7,8]} ## clf에서 사용되고 있던 파라미터 가져와봄


model = RandomizedSearchCV(estimator = clf, 
                           param_distributions = random_search,
                           cv =3,
                           verbose = 1,
                           random_state = 100)

for slicer in slicing_dic:
    i=0
    print(f'<<<---{slicer} start--->>>')
    print(datetime.now().strftime('%H:%M:%S'))
    tmpX = X[slicing_dic[slicer][0]:slicing_dic[slicer][1]].reset_index(drop = True) # X를 연속 4년치 꺼내서 reset_index(drop = True)
    tmpy = y[slicing_dic[slicer][0]:slicing_dic[slicer][1]].reset_index(drop = True) # y를 연속 4년치 꺼내서 reset_index(drop = True)
    tmp_all = d1221[slicing_dic[slicer][0]:slicing_dic[slicer][1]].reset_index(drop = True) # 전체데이터

    X_train = tmpX[:-slicing_dic[slicer][2]] # 기준년도 앞
    X_test = tmpX[-slicing_dic[slicer][2]:] # 기준년도 이하
    y_train = tmpy[:-slicing_dic[slicer][2]] # 기준년도 앞
    y_test = tmpy[-slicing_dic[slicer][2]:] # 기준년도 이하
    
    # train끼리 idx 같아야 하므로
    len_train = slicing_dic[slicer][1]-(slicing_dic[slicer][2]+slicing_dic[slicer][0]) # 혹은 tmp_train.shape[0]
    len(list(range(len_train)))
    train_idx = list(range(len_train))
    random.shuffle(train_idx) # 자동으로 덮어쓰기
    
    # test끼리 idx 같아야 하므로
    len_train_to_test = slicing_dic[slicer][1]-slicing_dic[slicer][0]
    test_idx = list(range(len_train,len_train_to_test)) # +1 안해줘도 되는 거 위 cell에서 확인
    len(test_idx)
    random.shuffle(test_idx) # 자동으로 덮어쓰기
    
    X_train = X_train.loc[train_idx]
    y_train = y_train.loc[train_idx]
    X_test = X_test.loc[test_idx]
    y_test = y_test.loc[test_idx]
    ###########################################
    y_train = np.array(y_train).reshape(-1,1) # 1열짜리로 만드는 것
    y_test = np.array(y_test).reshape(-1,1)
    model.fit(X_train,y_train)
    
    predictiontree = model.best_estimator_.predict(X_test)
#     print(confusion_matrix(y_test,predictiontree))
#     print(classification_report(y_test,predictiontree))
    print('best :', model.best_estimator_)
    acc3=accuracy_score(y_test,predictiontree)
    print(' acc:  ',acc3)
    ran_list.append([i,model.best_estimator_,acc3])
    i+=1
time.time()-start
ran_list
# 23:00:16 start
# 01:48:-- 미완
# randomforest max_leaf_nodes 5가지로 조정 16:48:28 -> 텀 47분~52분 소요

<<<---1215 start--->>>
19:12:41


297212

158287

Fitting 3 folds for each of 5 candidates, totalling 15 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(),
                   param_distributions={'max_leaf_nodes': [4, 5, 6, 7, 8]},
                   random_state=100, verbose=1)

...

Fitting 3 folds for each of 5 candidates, totalling 15 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(),
                   param_distributions={'max_leaf_nodes': [4, 5, 6, 7, 8]},
                   random_state=100, verbose=1)

best : RandomForestClassifier(max_leaf_nodes=8)
 acc:   0.5553746947974707


21589.794280052185

[[0, RandomForestClassifier(max_leaf_nodes=8), 0.5461787765261834],
 [0, RandomForestClassifier(max_leaf_nodes=8), 0.553233778995325],
 [0, RandomForestClassifier(max_leaf_nodes=8), 0.5563629470491596],
 [0, RandomForestClassifier(max_leaf_nodes=8), 0.5563802687881944],
 [0, RandomForestClassifier(max_leaf_nodes=8), 0.5555389701986013],
 [0, RandomForestClassifier(max_leaf_nodes=8), 0.5312484534366325],
 [0, RandomForestClassifier(max_leaf_nodes=8), 0.5553746947974707]]

In [15]:
ran_list

[[0, RandomForestClassifier(max_leaf_nodes=8), 0.5461787765261834],
 [0, RandomForestClassifier(max_leaf_nodes=8), 0.553233778995325],
 [0, RandomForestClassifier(max_leaf_nodes=8), 0.5563629470491596],
 [0, RandomForestClassifier(max_leaf_nodes=8), 0.5563802687881944],
 [0, RandomForestClassifier(max_leaf_nodes=8), 0.5555389701986013],
 [0, RandomForestClassifier(max_leaf_nodes=8), 0.5312484534366325],
 [0, RandomForestClassifier(max_leaf_nodes=8), 0.5553746947974707]]

# XGBoost 하이퍼파라미터 찾기
learning rate

In [16]:
l1=[]
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import time
import warnings
warnings.filterwarnings('ignore')
start = time.time()
# X, y
X = d1221.iloc[:,2:-1] # 1221 데이터
y = d1221.iloc[:,-1]

# seed 고정
user_seed = 0
random.seed(user_seed) # seed 고정

clf = XGBClassifier()
random_search = {'learning_rate': [0.001, 0.005, 0.05,0.2]}


model = RandomizedSearchCV(estimator = clf, 
                           param_distributions = random_search,
                           cv =3,
                           verbose = 1,
                           random_state = 100)

for slicer in slicing_dic:
    i=0
    print(f'<<<---{slicer} start--->>>')
    print(datetime.now().strftime('%H:%M:%S'))
    tmpX = X[slicing_dic[slicer][0]:slicing_dic[slicer][1]].reset_index(drop = True) # X를 연속 4년치 꺼내서 reset_index(drop = True)
    tmpy = y[slicing_dic[slicer][0]:slicing_dic[slicer][1]].reset_index(drop = True) # y를 연속 4년치 꺼내서 reset_index(drop = True)
    tmp_all = d1221[slicing_dic[slicer][0]:slicing_dic[slicer][1]].reset_index(drop = True) # 전체데이터

    X_train = tmpX[:-slicing_dic[slicer][2]] # 기준년도 앞
    X_test = tmpX[-slicing_dic[slicer][2]:] # 기준년도 이하
    y_train = tmpy[:-slicing_dic[slicer][2]] # 기준년도 앞
    y_test = tmpy[-slicing_dic[slicer][2]:] # 기준년도 이하
    
    # train끼리 idx 같아야 하므로
    len_train = slicing_dic[slicer][1]-(slicing_dic[slicer][2]+slicing_dic[slicer][0]) # 혹은 tmp_train.shape[0]
    len(list(range(len_train)))
    train_idx = list(range(len_train))
    random.shuffle(train_idx) # 자동으로 덮어쓰기
    
    # test끼리 idx 같아야 하므로
    len_train_to_test = slicing_dic[slicer][1]-slicing_dic[slicer][0]
    test_idx = list(range(len_train,len_train_to_test)) # +1 안해줘도 되는 거 위 cell에서 확인
    len(test_idx)
    random.shuffle(test_idx) # 자동으로 덮어쓰기
    
    X_train = X_train.loc[train_idx]
    y_train = y_train.loc[train_idx]
    X_test = X_test.loc[test_idx]
    y_test = y_test.loc[test_idx]
    ###########################################
    y_train = np.array(y_train).reshape(-1,1) # 1열짜리로 만드는 것
    y_test = np.array(y_test).reshape(-1,1)
    model.fit(X_train,y_train)
    
    predictiontree = model.best_estimator_.predict(X_test)
#     print(confusion_matrix(y_test,predictiontree))
#     print(classification_report(y_test,predictiontree))
    print('best :', model.best_estimator_)
    acc3=accuracy_score(y_test,predictiontree)
    print(' acc:  ',acc3)
    l1.append([i, model.best_estimator_, acc3])
    i+=1
time.time()-start


<<<---1215 start--->>>
01:12:30


297212

158287

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[01:12:35] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:14:32] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:16:25] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:18:17] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Ex

RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           enable_categorical=False, gamma=None,
                                           gpu_id=None, importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100, n_jobs=None,
                                           num_parallel_tree=None,
                                     

...

223622

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[03:54:36] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:56:43] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:58:56] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[04:01:13] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Ex

RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           enable_categorical=False, gamma=None,
                                           gpu_id=None, importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100, n_jobs=None,
                                           num_parallel_tree=None,
                                     

best : XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.2, max_delta_step=0,
              max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=128,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
 acc:   0.553143250664067


11891.889689922333

In [17]:
l1

[[0,
  XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
                gamma=0, gpu_id=-1, importance_type=None,
                interaction_constraints='', learning_rate=0.05, max_delta_step=0,
                max_depth=6, min_child_weight=1, missing=nan,
                monotone_constraints='()', n_estimators=100, n_jobs=128,
                num_parallel_tree=1, predictor='auto', random_state=0,
                reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
                tree_method='exact', validate_parameters=1, verbosity=None),
  0.5584160417469534],
 [0,
  XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
                gamma=0, gpu_id=-1, importance_type=None,
                interaction_constraints='', learning_rate=0.05, max_delta_step=0,
                max_de

# MLP 하이퍼파라미터 찾기
learning rate init

In [18]:
l2=[]
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import time
import warnings
warnings.filterwarnings('ignore')
start = time.time()
# X, y
X = d1221.iloc[:,2:-1] # 1221 데이터
y = d1221.iloc[:,-1]

# seed 고정
user_seed = 0
random.seed(user_seed) # seed 고정

clf = MLPClassifier()
random_search = {'learning_rate_init': [0.005, 0.05, 0.0005, 0.001 ]}


model = RandomizedSearchCV(estimator = clf, 
                           param_distributions = random_search,
                           cv =3,
                           verbose = 1,
                           random_state = 100)

for slicer in slicing_dic:
    i=0
    print(f'<<<---{slicer} start--->>>')
    print(datetime.now().strftime('%H:%M:%S'))
    tmpX = X[slicing_dic[slicer][0]:slicing_dic[slicer][1]].reset_index(drop = True) # X를 연속 4년치 꺼내서 reset_index(drop = True)
    tmpy = y[slicing_dic[slicer][0]:slicing_dic[slicer][1]].reset_index(drop = True) # y를 연속 4년치 꺼내서 reset_index(drop = True)
    tmp_all = d1221[slicing_dic[slicer][0]:slicing_dic[slicer][1]].reset_index(drop = True) # 전체데이터

    X_train = tmpX[:-slicing_dic[slicer][2]] # 기준년도 앞
    X_test = tmpX[-slicing_dic[slicer][2]:] # 기준년도 이하
    y_train = tmpy[:-slicing_dic[slicer][2]] # 기준년도 앞
    y_test = tmpy[-slicing_dic[slicer][2]:] # 기준년도 이하
    
    # train끼리 idx 같아야 하므로
    len_train = slicing_dic[slicer][1]-(slicing_dic[slicer][2]+slicing_dic[slicer][0]) # 혹은 tmp_train.shape[0]
    len(list(range(len_train)))
    train_idx = list(range(len_train))
    random.shuffle(train_idx) # 자동으로 덮어쓰기
    
    # test끼리 idx 같아야 하므로
    len_train_to_test = slicing_dic[slicer][1]-slicing_dic[slicer][0]
    test_idx = list(range(len_train,len_train_to_test)) # +1 안해줘도 되는 거 위 cell에서 확인
    len(test_idx)
    random.shuffle(test_idx) # 자동으로 덮어쓰기
    
    X_train = X_train.loc[train_idx]
    y_train = y_train.loc[train_idx]
    X_test = X_test.loc[test_idx]
    y_test = y_test.loc[test_idx]
    ###########################################
    y_train = np.array(y_train).reshape(-1,1) # 1열짜리로 만드는 것
    y_test = np.array(y_test).reshape(-1,1)
    model.fit(X_train,y_train)
    
    predictiontree = model.best_estimator_.predict(X_test)
#     print(confusion_matrix(y_test,predictiontree))
#     print(classification_report(y_test,predictiontree))
    print('best :', model.best_estimator_)
    acc3=accuracy_score(y_test,predictiontree)
    print(' acc:  ',acc3)
    l2.append([i, model.best_estimator_, acc3])
    i+=1
time.time()-start
l2

<<<---1215 start--->>>
04:30:42


297212

158287

Fitting 3 folds for each of 4 candidates, totalling 12 fits


RandomizedSearchCV(cv=3, estimator=MLPClassifier(),
                   param_distributions={'learning_rate_init': [0.005, 0.05,
                                                               0.0005, 0.001]},
                   random_state=100, verbose=1)

...

Fitting 3 folds for each of 4 candidates, totalling 12 fits


RandomizedSearchCV(cv=3, estimator=MLPClassifier(),
                   param_distributions={'learning_rate_init': [0.005, 0.05,
                                                               0.0005, 0.001]},
                   random_state=100, verbose=1)

best : MLPClassifier(learning_rate_init=0.05)
 acc:   0.5520029335217466


11107.739107608795

[[0, MLPClassifier(), 0.5410109484670251],
 [0, MLPClassifier(learning_rate_init=0.005), 0.5444272399726052],
 [0, MLPClassifier(), 0.5533841237746204],
 [0, MLPClassifier(), 0.5490205920367419],
 [0, MLPClassifier(), 0.5522476061110406],
 [0, MLPClassifier(learning_rate_init=0.05), 0.5321977621509266],
 [0, MLPClassifier(learning_rate_init=0.05), 0.5520029335217466]]

# Support Vector Machine 하이퍼 파라미터
C , degree

In [ ]:
svc_list=[]
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import time
import warnings
warnings.filterwarnings('ignore')
start = time.time()
# X, y
X = d1221.iloc[:,2:-1] # 1221 데이터
y = d1221.iloc[:,-1]

# seed 고정
user_seed = 0
random.seed(user_seed) # seed 고정

clf = SVC()
random_search = {'C': [0.2, 1.0, 10.0],
                'gamma': [0.5, 1, 3]}


model = RandomizedSearchCV(estimator = clf, 
                           param_distributions = random_search,
                           cv =3,
                           verbose = 1,
                           random_state = 100)

for slicer in slicing_dic:
    i=0
    print(f'<<<---{slicer} start--->>>')
    print(datetime.now().strftime('%H:%M:%S'))
    tmpX = X[slicing_dic[slicer][0]:slicing_dic[slicer][1]].reset_index(drop = True) # X를 연속 4년치 꺼내서 reset_index(drop = True)
    tmpy = y[slicing_dic[slicer][0]:slicing_dic[slicer][1]].reset_index(drop = True) # y를 연속 4년치 꺼내서 reset_index(drop = True)
    tmp_all = d1221[slicing_dic[slicer][0]:slicing_dic[slicer][1]].reset_index(drop = True) # 전체데이터

    X_train = tmpX[:-slicing_dic[slicer][2]] # 기준년도 앞
    X_test = tmpX[-slicing_dic[slicer][2]:] # 기준년도 이하
    y_train = tmpy[:-slicing_dic[slicer][2]] # 기준년도 앞
    y_test = tmpy[-slicing_dic[slicer][2]:] # 기준년도 이하
    
    # train끼리 idx 같아야 하므로
    len_train = slicing_dic[slicer][1]-(slicing_dic[slicer][2]+slicing_dic[slicer][0]) # 혹은 tmp_train.shape[0]
    len(list(range(len_train)))
    train_idx = list(range(len_train))
    random.shuffle(train_idx) # 자동으로 덮어쓰기
    
    # test끼리 idx 같아야 하므로
    len_train_to_test = slicing_dic[slicer][1]-slicing_dic[slicer][0]
    test_idx = list(range(len_train,len_train_to_test)) # +1 안해줘도 되는 거 위 cell에서 확인
    len(test_idx)
    random.shuffle(test_idx) # 자동으로 덮어쓰기
    
    X_train = X_train.loc[train_idx]
    y_train = y_train.loc[train_idx]
    X_test = X_test.loc[test_idx]
    y_test = y_test.loc[test_idx]
    ###########################################
    y_train = np.array(y_train).reshape(-1,1) # 1열짜리로 만드는 것
    y_test = np.array(y_test).reshape(-1,1)
    model.fit(X_train,y_train)
    
    predictiontree = model.best_estimator_.predict(X_test)
#     print(confusion_matrix(y_test,predictiontree))
#     print(classification_report(y_test,predictiontree))
    print('best :', model.best_estimator_)
    acc3=accuracy_score(y_test,predictiontree)
    print(' acc:  ',acc3)
    svc_list.append([i, model.best_estimator_, acc3])
    i+=1
time.time()-start
svc_list

<<<---1215 start--->>>
07:35:50


297212

158287

Fitting 3 folds for each of 9 candidates, totalling 27 fits
